<a href="https://colab.research.google.com/github/InstitutodaVinci3/Vancouver/blob/master/Producao_Diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Resumo dos Trabalhos de Coleta, Limpeza e Entendimento dos dados

##Biblotecas Utilizadas

In [121]:
# importando as bibliotecas básicas e preparando ambiente
import pandas as pd                     # convenção 
import numpy as np                      # convenção 
import random as rd                     # rd não é alias padrão mas sim adotado aqui somente 
# Itens adicionais
import glob
import os
import re

##Coleta e Limpeza dos Dados

###Passo 1 - montagem do drive para colocação dos arquivos recebidos

In [ ]:
# montagem do Google Drive para acessar os arquivos que foram disponibilizados
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Passo 2 - definição da varíavel "Filenames" para ser utilizado no "for" de leitura, de forma a correr todoos os arquivos do tipo TXT existentes no caminho (path)

In [122]:
# Defifinindo a varíavel FIlenames para ser utilizado no "for" de leitura
#path =r'https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/'
filenames = glob.glob('/content/drive/MyDrive/MedMep/ProducaoDiaria/*.xlsx')
filenames

['/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170903T172000.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170904T172000.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170901T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170902T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170922T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170916T172000.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170915T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170919T172000.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170917T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170920T172001.xlsx',
 '/content/drive/MyDrive/MedMep/ProducaoDiaria/Produção diária_20170918T172000.xlsx',
 '/content/drive/MyDr

###Passo 3 - Definição de como capturar a data que está no nome do arquivo
- É necessário capturar a data inserindo-a em uma coluna adicional para que possamos utilizar os dados de produção de leite em conjunto com os dados de temperatura
- Foi necessário tratar da o formato da data capturado

In [123]:
# Ler parte do nome do arquivo
b = re.search(r'(\d+)', '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária_20170901T173500.xlsx')
#b = re.search(r'(\d+)', '/content/drive/MyDrive/MedMep/ProducaoDiaria/JULHO/*.xlsx')
print(b[1])

#Formatando Data
import datetime
datetime.datetime.strptime("20170901", "%Y%m%d").strftime("%d/%m/%Y")

20170901


'01/09/2017'

###Passo 4 - Leitura dos arquivos
- Foram disbonibilizados 177 arquivos no formato TXT, Nomenclatura: Produção diária TXT_AAAAMMDDTHHMMSS
- Foi necessário descartar a primeira linha (cabeçalho) de todos os arquivos por falta de padrão nos arquivos enviados
- Foi necessário remover as duas últimas linhas que comtinham comnandos de "end line" e não seriam necessários ao ralizarmos a concatenação dos arquivos
- Foi necessário criar uma nova definição para o cabeçalho do DataFrame
- Foi necessário criar uma nova coluna para a inserção da data da coleta que estava no nome do arquivo
- Os 177 arquivos foram lidos e concatenados gerando um único DataFrame unificado com todos os dados da Produção Diária de leit

In [94]:
for filename in filenames:
# Leitura dos arquivos já com o novo cabeçalho definido:
  a = pd.read_excel(filename, sheet_name=1, na_values=['NaN','NA','N/A','NaT','',' '],
                  names=['Grupo', 'Id_Animal', 'Num_Lac', 'Dias_Lac', 'Pico', 'Hora_S1', 'Hora_S2', 'Hora_S3',
                         'Durac_S1', 'Durac_S2', 'Durac_S3', 'Prod_S1', 'Prod_S2', 'Prod_S3','Hora_S1_ont', 
                         'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', 'Dura_S3_ont','Prod_S1_ont', 
                         'Prod_S2_ont', 'Prod_S3_ont'])

In [130]:
# Lendo os dados já com removendo o cabeçalho e rodapés dos arquivos
# Necessário criar o "Cabeçalho" do DataFrame conforme comando asseguir:
Prod_Dia_df = pd.DataFrame({'Grupo':[], 'Id_Animal':[], 'Num_Lac':[], 'Dias_Lac':[], 'Pico':[], 'Hora_S1':[],
                            'Hora_S2':[], 'Hora_S3':[], 'Durac_S1':[], 'Durac_S2':[], 'Durac_S3':[],
                            'Prod_S1':[], 'Prod_S2':[], 'Prod_S3':[], 'Hora_S1_ont':[], 'Hora_S2_ont':[], 
                            'Hora_S3_ont':[], 'Dura_S1_ont':[], 'Dura_S2_ont':[], 'Dura_S3_ont':[],
                            'Prod_S1_ont':[], 'Prod_S2_ont':[], 'Prod_S3_ont':[]})

# Início do looping de leitura dos arquivos
for filename in filenames:

# Leitura dos arquivos já com o novo cabeçalho definido:
  a = pd.read_excel(filename, sheet_name=1, na_values=['NaN','NA','N/A','NaT','',' '], skipfooter=0,
                  names=['Grupo', 'Id_Animal', 'Num_Lac', 'Dias_Lac', 'Pico', 'Hora_S1', 'Hora_S2', 'Hora_S3',
                         'Durac_S1', 'Durac_S2', 'Durac_S3', 'Prod_S1', 'Prod_S2', 'Prod_S3','Hora_S1_ont', 
                         'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', 'Dura_S3_ont','Prod_S1_ont', 
                         'Prod_S2_ont', 'Prod_S3_ont'])
# Criação da coluna adicional com a data contida no nome do arquivo:
  a['Data']= datetime.datetime.strptime(re.search(r'(\d+)', filename)[1], "%Y%m%d").strftime("%d/%m/%Y") 

# Concatenando os arquivos lidos dentro do looping 
  Prod_Dia_df = pd.concat([Prod_Dia_df, a])
# Fim do looping de leitura

Prod_Dia_df.head()

,Grupo,Id_Animal,Num_Lac,Dias_Lac,Pico,Hora_S1,Hora_S2,Hora_S3,Durac_S1,Durac_S2,Durac_S3,Prod_S1,Prod_S2,Prod_S3,Hora_S1_ont,Hora_S2_ont,Hora_S3_ont,Dura_S1_ont,Dura_S2_ont,Dura_S3_ont,Prod_S1_ont,Prod_S2_ont,Prod_S3_ont,Data
0,NaN,4426.0,2.0,91.0,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,03/09/2017
1,NaN,2646.0,1.0,493.0,11.24,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,03/09/2017
2,1,3652.0,2.0,93.0,NaN,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,1901-01-05 00:00:00,1900-10-08 00:00:00,NaN,12.58,5.96,NaN,2017-09-02 08:00:00,2017-09-02 15:30:00,NaN,1901-01-30,1900-08-23 00:00:00,NaN,12.49,6.63,NaN,03/09/2017
3,1,653.0,2.0,128.0,NaN,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,1901-08-02 00:00:00,1901-01-14 00:00:00,NaN,18.19,8.54,NaN,2017-09-02 08:00:00,2017-09-02 15:30:00,NaN,1901-03-10,1900-12-27 00:00:00,NaN,16.06,8.86,NaN,03/09/2017
4,1,2623.0,1.0,94.0,NaN,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,1901-01-16 00:00:00,1900-11-24 00:00:00,NaN,12.94,7.95,NaN,2017-09-02 08:00:00,2017-09-02 15:30:00,NaN,1901-01-26,1900-11-23 00:00:00,NaN,12.49,8.20,NaN,03/09/2017


In [131]:
Prod_Dia_limpo_df.tail(40)

,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
17,4557.0,2017-08-01 09:00:00,2017-08-01 16:30:00,NaN,14.19,7.12,NaN,01/08/2017
18,4560.0,NaT,NaT,NaN,NaN,NaN,NaN,01/08/2017
19,4525.0,NaT,NaT,NaN,NaN,NaN,NaN,01/08/2017
20,4535.0,NaT,NaT,NaN,NaN,NaN,NaN,01/08/2017
21,4066.0,NaT,NaT,NaN,NaN,NaN,NaN,01/08/2017
22,5407.0,2017-08-01 09:30:00,2017-08-01 16:30:00,NaN,12.53,8.24,NaN,01/08/2017
23,4583.0,2017-08-01 09:00:00,2017-08-01 16:30:00,NaN,11.57,6.40,NaN,01/08/2017
24,4587.0,NaT,2017-08-01 17:00:00,NaN,NaN,3.33,NaN,01/08/2017
25,4573.0,2017-08-01 09:00:00,2017-08-01 16:30:00,NaN,2.61,8.84,NaN,01/08/2017
26,4581.0,2017-08-01 09:30:00,2017-08-01 16:30:00,NaN,11.89,10.13,NaN,01/08/2017


In [132]:
# Exportando dados dos arquivos unificados mas não limpos para CSV
Prod_Dia_df.to_csv("/content/drive/MyDrive/MedMep/ProducaoDiaria/Prod_Diaria_Unif_Sujo.csv")

###Limpeza do arquivo unificado
- Remoção das colunas com dados não pertinentes ou que estavam em branco:
- Informações sobre o Grupo não dizem nada ao nosso estudo (não se sabe o critério utilizado no agrupamento), 
- Numero de dias de lactação. Dado existe somente para poucos animais,
- Pico de produção. Dado existe somente para poucos animais e não se sabe o seu significado,
- Dura_Sx. Refere-se a duração da sessão de ordenha. Dado existe somente para poucas sessões de ordenha,
- Dura_Sx_ont. Refere-se a duração da sessão de ordenha do dia anterior. Dado existe somente para poucas sessões de ordenha,
- Prod_Sx_ont. Refere-se a produção da sessão Sx do dia anterior. Dado existe somente para poucas sessões e animais.

In [98]:
Prod_Dia_limpo_df = Prod_Dia_df.drop(columns=['Grupo', 'Num_Lac', 'Dias_Lac', 'Pico', 'Durac_S1', 'Durac_S2', 'Durac_S3', 
                                              'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
                                              'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'])
# Prod_Dia_limpo_df.head()

# Prod_Dia_df.drop(columns=['Grupo', 'Num_Lac', 'Dias_Lac', 'Pico', 'Durac_S1', 'Durac_S2', 'Durac_S3', 
#          'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
#          'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'], inplace=True)
# Prod_Dia_df.head()

In [106]:
# Exibindo as primeiras linhas do arquivo
Prod_Dia_limpo_df.head()

,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,4426.0,NaT,NaT,NaN,NaN,NaN,NaN,03/09/2017
1,2646.0,NaT,NaT,NaN,NaN,NaN,NaN,03/09/2017
2,3652.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.58,5.96,NaN,03/09/2017
3,653.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,18.19,8.54,NaN,03/09/2017
4,2623.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.94,7.95,NaN,03/09/2017


- Hora_Sx refere-se a hora que se iniciou a sessão de ordenha X no formato HH:00
- Prod_Sx refere-se a produção de leite da sessão X em KG

In [100]:
# Exibindo as últimas linhas do arquivo
Prod_Dia_limpo_df.tail()

,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
52,3460.0,2017-08-01 07:30:00,2017-08-01 15:30:00,NaN,20.00,10.70,NaN,01/08/2017
53,2483.0,2017-08-01 08:00:00,2017-08-01 15:30:00,NaN,13.69,6.51,NaN,01/08/2017
54,2431.0,2017-08-01 08:00:00,2017-08-01 16:00:00,NaN,31.83,7.91,NaN,01/08/2017
55,55.0,NaT,NaT,NaN,513.19,254.05,NaN,01/08/2017
56,NaN,NaT,NaT,NaN,NaN,NaN,NaN,01/08/2017


###Salvando os dados limpos em arquivo para utilização futura
- Será salvo no formato CSV de maneira a facilitar a leitura nas próximas etapas

In [107]:
# Exportando dados dos arquivos unificados e limpos para CSV
Prod_Dia_limpo_df.to_csv("/content/drive/MyDrive/MedMep/ProducaoDiaria/Prod_Diaria_Unificado.csv")

##Entendendo os dados


###Estatística descritiva - análise inicial

In [110]:
df=pd.read_csv('https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/Prod_Diaria_Unificado.csv')
df.head()

,Unnamed: 0,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,0,4426.0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2017
1,1,2646.0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2017
2,2,3652.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.58,5.96,NaN,03/09/2017
3,3,653.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,18.19,8.54,NaN,03/09/2017
4,4,2623.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.94,7.95,NaN,03/09/2017


In [111]:
df.shape

(6302, 9)

In [112]:
df.describe()

,Unnamed: 0,Id_Animal,Prod_S1,Prod_S2,Prod_S3
count,6302.000000,6210.000000,4141.000000,3137.000000,751.000000
mean,34.350682,3977.403382,22.946916,12.405827,6.394780
std,20.743286,1009.400379,75.865345,39.845768,2.358338
min,0.000000,55.000000,0.000000,0.000000,0.010000
25%,17.000000,3475.000000,9.520000,5.110000,5.320000
50%,34.000000,4475.000000,11.910000,6.380000,6.560000
75%,51.000000,4572.000000,14.090000,7.640000,7.595000
max,75.000000,5409.000000,699.320000,417.880000,18.210000


- A função Describe não realizou as estastíticas para as colunas Prod_Sx (ñ estão sendo entendidos como numéricos?)

In [119]:
df.Prod_S1.describe()

count    4141.000000
mean       22.946916
std        75.865345
min         0.000000
25%         9.520000
50%        11.910000
75%        14.090000
max       699.320000
Name: Prod_S1, dtype: float64

In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
df.Prod_S2.describe()

count    3137.000000
mean       12.405827
std        39.845768
min         0.000000
25%         5.110000
50%         6.380000
75%         7.640000
max       417.880000
Name: Prod_S2, dtype: float64

In [115]:
df.Prod_S3.describe()

count    751.000000
mean       6.394780
std        2.358338
min        0.010000
25%        5.320000
50%        6.560000
75%        7.595000
max       18.210000
Name: Prod_S3, dtype: float64

###Dashboard

In [116]:
df=pd.read_csv('https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/Prod_Diaria_Unificado.csv')
df.head()

,Unnamed: 0,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,0,4426.0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2017
1,1,2646.0,NaN,NaN,NaN,NaN,NaN,NaN,03/09/2017
2,2,3652.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.58,5.96,NaN,03/09/2017
3,3,653.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,18.19,8.54,NaN,03/09/2017
4,4,2623.0,2017-09-03 08:00:00,2017-09-03 15:30:00,NaN,12.94,7.95,NaN,03/09/2017


In [ ]:
# Producao_S1 = Prod_Dia_df.groupby('Id_Animal').Prod_S1.sum()
# Producao_S1

In [ ]:
# Exportando dados dos arquivos unificados para Excel
# Producao_S1.to_excel('/content/drive/MyDrive/MedMep/Prod_Diaria/Producao-S1.xlsx')